In [1]:
import folium # map rendering library
import geocoder # import geocoder
import pandas as pd
import numpy as np
import requests # library to handle requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
wiki_coord = pd.read_csv("wiki_coord.csv")

Let's start by creting a map of ou neighborhoods

Using the coordiantes in the data set let's make a map indicating the locatio of all neighberhoods

In [3]:
# create map using latitude and longitude values
map_wiki = folium.Map(location=[wiki_coord['latitude'].iloc[50], wiki_coord['longitude'].iloc[50]],zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(wiki_coord['latitude'], wiki_coord['longitude'], wiki_coord['Borough'], wiki_coord['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_wiki)  
    
map_wiki

In [4]:
wiki_coord.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...,43.625490,-79.526000
1,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.632798,-79.493017
2,M7Y,East Toronto,Business reply mail Processing CentrE,43.648700,-79.385450
3,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
4,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653340,-79.509766


getNearbyVenues will be function to collect venues information of a given neighborhood

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'P1NLVQZVKMATOQZBPZYDKCGKXUXIUIFUKUDIUTEP0HA42GUW', 
            'PNP0LXC4CKO2OMZZUZKIH5TF34D4EVXC1VO5324DFN0AH5AF', 
            '20200420', 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's use the funnction to collect the data we need

In [6]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Toronto_venues = getNearbyVenues(names=wiki_coord['Neighborhood'],
                                   latitudes=wiki_coord['latitude'],
                                   longitudes=wiki_coord['longitude']
                                  )



Mimico NW / The Queensway West / South of Bloor / Kingsway Park South West / Royal York South West
Old Mill South / King's Mill Park / Sunnylea / Humber Bay / Mimico NE / The Queensway East / Royal York South East / Kingsway Park South East
Business reply mail Processing CentrE
Church and Wellesley
The Kingsway / Montgomery Road  / Old Mill North
First Canadian Place / Underground city
St. James Town / Cabbagetown
Upper Rouge
Northwest
Alderwood / Long Branch
Stn A PO Boxes
Rosedale
Steeles West / L'Amoreaux West
South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens
New Toronto / Mimico South / Humber Bay Shores
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Kensington Market / Chinatown / Grange Park
Moore Park / Summerhill East
C

A little bit of data wrangling to get everything in place

In [7]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,Alderwood / Long Branch,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Bedford Park / Lawrence Manor East,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Willowdale / Newtonbrook,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
93,York Mills / Silver Hills,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


let's get our neighborhood into 3 clusters 

In [9]:
# set number of clusters
kclusters = 3

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [10]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
              venue  freq
0  Sushi Restaurant   0.2
1    Breakfast Spot   0.2
2   Badminton Court   0.2
3      Skating Rink   0.2
4       Supermarket   0.2


----Alderwood / Long Branch----
               venue  freq
0        Pizza Place  0.12
1     Sandwich Place  0.12
2  Convenience Store  0.12
3           Pharmacy  0.12
4        Coffee Shop  0.12


----Bathurst Manor / Wilson Heights / Downsview North----
                 venue  freq
0                 Bank  0.09
1          Coffee Shop  0.09
2       Ice Cream Shop  0.05
3  Fried Chicken Joint  0.05
4          Bridal Shop  0.05


----Bayview Village----
                        venue  freq
0                       Trail   0.5
1  Construction & Landscaping   0.5
2                 Yoga Studio   0.0
3     New American Restaurant   0.0
4                 Music Venue   0.0


----Bedford Park / Lawrence Manor East----
                     venue  freq
0           Sandwich Place  0.09
1       Italian Restaurant  0.09
2         

                    venue  freq
0          Clothing Store  0.23
1          Cosmetics Shop  0.04
2             Men's Store  0.04
3  Furniture / Home Store  0.04
4        Sushi Restaurant  0.04


----Lawrence Park----
         venue  freq
0  Swim School  0.33
1     Bus Line  0.33
2       Lawyer  0.33
3  Yoga Studio  0.00
4  Music Venue  0.00


----Leaside----
                    venue  freq
0     Sporting Goods Shop  0.08
1             Coffee Shop  0.08
2  Furniture / Home Store  0.06
3                    Bank  0.06
4              Sports Bar  0.06


----Little Portugal / Trinity----
              venue  freq
0      Cocktail Bar  0.07
1        Restaurant  0.07
2               Bar  0.07
3          Wine Bar  0.05
4  Asian Restaurant  0.05


----Malvern / Rouge----
                     venue  freq
0                    Trail   1.0
1              Yoga Studio   0.0
2  New American Restaurant   0.0
3              Music Venue   0.0
4                   Museum   0.0


----Milliken / Agincourt North

                     venue  freq
0              Auto Garage   1.0
1              Yoga Studio   0.0
2             Night Market   0.0
3  New American Restaurant   0.0
4              Music Venue   0.0


----Willowdale----
               venue  freq
0        Coffee Shop  0.07
1   Ramen Restaurant  0.07
2        Pizza Place  0.05
3               Park  0.05
4  Korean Restaurant  0.05


----Willowdale / Newtonbrook----
                     venue  freq
0              Bridal Shop  0.33
1                Piano Bar  0.33
2                     Café  0.33
3      Monument / Landmark  0.00
4  New American Restaurant  0.00


----Woburn----
               venue  freq
0        Coffee Shop  0.67
1  Korean Restaurant  0.33
2      Movie Theater  0.00
3       Noodle House  0.00
4       Night Market  0.00


----Woodbine Heights----
         venue  freq
0  Pizza Place  0.08
1          Pub  0.08
2   Smoke Shop  0.08
3  Gas Station  0.08
4         Café  0.08


----York Mills / Silver Hills----
            venue 

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [12]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Sushi Restaurant,Badminton Court,Supermarket,Skating Rink,Food,Flower Shop,Fish Market,Fish & Chips Shop,Field
1,Alderwood / Long Branch,Pizza Place,Gym,Convenience Store,Pub,Coffee Shop,Sandwich Place,Gas Station,Pharmacy,Gym / Fitness Center,Discount Store
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Chinese Restaurant,Bridal Shop,Convenience Store,Sandwich Place,Middle Eastern Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant
3,Bayview Village,Construction & Landscaping,Trail,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Eastern European Restaurant,Field
4,Bedford Park / Lawrence Manor East,Italian Restaurant,Sandwich Place,Coffee Shop,Fast Food Restaurant,Spa,Juice Bar,Pub,Restaurant,Sports Club,Café


In [13]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = wiki_coord

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...,43.625490,-79.526000,0.0,Burrito Place,Bank,Coffee Shop,Burger Joint,Pet Store,Fish & Chips Shop,Middle Eastern Restaurant,Miscellaneous Shop,Sushi Restaurant,Thai Restaurant
1,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.632798,-79.493017,2.0,Home Service,Park,Baseball Field,Tennis Court,Bank,Food,Flower Shop,Fish Market,Fish & Chips Shop,Electronics Store
2,M7Y,East Toronto,Business reply mail Processing CentrE,43.648700,-79.385450,0.0,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,American Restaurant,Asian Restaurant,Sandwich Place,Theater,Sushi Restaurant
3,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472,0.0,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Pub,Grocery Store,Dance Studio,Gay Bar,Burger Joint,Gastropub
4,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653340,-79.509766,2.0,Smoke Shop,Pool,Park,Women's Store,Farmers Market,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm


In [15]:
Toronto_merged.dropna(inplace = True)

And now for the map with our clusters

In [16]:
# create map
map_clusters = folium.Map(location=[wiki_coord['latitude'].iloc[50], wiki_coord['longitude'].iloc[50]], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['latitude'], Toronto_merged['longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters